In [8]:
import networkx as nx

from csv import reader
from collections import defaultdict
import matplotlib.pyplot as plt

from statistics import mean

In [15]:
EDGE_DATA = 'conversation_edgelist27min.csv' #input the necessary edge list
EDGE_OUTPUT_FILE = 'conversations_edges_27min.csv'
GRAPH_OUTPUT_FILE = 'conversations_graph_27min.csv'

In [16]:
m_graphs_nodes = defaultdict(set)
m_graphs_edges = defaultdict(list)
u_graphs_nodes = defaultdict(set)
u_graphs_edges = defaultdict(list)

with open(EDGE_DATA, 'r') as file:
    csv_reader = reader(file)
    
    for n, row in enumerate(csv_reader):
        if n == 0:
            continue
        
        g_id = row[14]
        
        m_edge = [ row[1], row[2], row[3] ]
        m_graphs_nodes[g_id].add( row[1] )
        m_graphs_nodes[g_id].add( row[2] )
        m_graphs_edges[g_id].append(m_edge)

In [17]:
m_graphs = dict()
u_graphs = dict()

edges_output_file = open(EDGE_OUTPUT_FILE, 'w')
graph_output_file = open(GRAPH_OUTPUT_FILE, 'w')

graph_uid = 0
for key, value in m_graphs_edges.items():
    g = nx.DiGraph()
    
    for edge in value:
        g.add_edge(edge[0], edge[1], weight = edge[2])
        
    for component in nx.weakly_connected_components(g):
        c = nx.subgraph(g, component)        
        
        n_nodes = c.order()
        n_edges = c.size()
        deg_c = mean(nx.degree_centrality(c).values())
        ideg_c = mean(nx.in_degree_centrality(c).values())
        odeg_c = mean(nx.out_degree_centrality(c).values())
        
        clu = nx.average_clustering(c)
        recip = nx.overall_reciprocity(c)
        dens = nx.density(c)
        
        for u, v, weight in c.edges(data = "weight"):
            edges_output_file.write(f"{graph_uid}, {u}, {v}, {weight}\n")
                              
        graph_output_file.write(f"{graph_uid}, {n_nodes}, {n_edges}, {ideg_c}, {odeg_c}, {deg_c}, {clu}, {recip}, {dens}\n")
        graph_uid = graph_uid + 1